In [6]:
import numpy as np
from gru import (
    # GRUModel,
    fibonacci_generator,
    one_by_n_generator,
    plus_one_minus_one_generator,
    squared_generator,
    half_generator,
    arithmetic_progression,
    create_sliding_window_batches,
    mape
)
from gru2 import GRUModel
import plotly.graph_objects as go

In [7]:
# seq_length = 10
# window_size = 5
# batch_size = 2
# hidden_size = 4
# output_size = 3

# sequence = list(fibonacci_generator(seq_length))
# X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)
# X_train, y_train = X[0:int(len(X) * 0.8)], y[0:int(len(X) * 0.8)][-1]
# X_test, y_test = X[int(len(X) * 0.8):], y[int(len(X) * 0.8):][-1]

# verbosity = 1000

# max_mape = 1e-3
# learn_until_meet_mape = True

# model = GRUModel(window_size, hidden_size, output_size)

# # Параметры обучения
# lr = 1e-4
# epochs = 5000

# model.train(X_train, y_train, lr, epochs,
#             True, max_mape, verbosity)
# Проверим результат после обучения
# y_pred, _, _ = model.forward(X_train)
# print("Предсказание после обучения:\n", y_pred)
# print("Истинное значение:\n", y_train)
# print(f"MAPE: {mape(y_train, y_pred) * 100:.4f}%")

# print('TEST')
# y_pred, _, _ = model.forward(X_test)
# print("Предсказания на тестовой выборке:\n", y_pred)
# print("Истинное значение на тестовой выборке:\n", y_test)
# print(f"MAPE: {mape(y_test, y_pred) * 100:.4f}%")  

In [8]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
epochs = list(range(100, 2100, 100))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_epochs: list[int] = []

seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

verbosity = 100000
lr = 1e-4

sequence = list(arithmetic_progression(seq_length, a0=1, d=1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

for max_epochs in epochs:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):              
        
        network = GRUModel(input_size=window_size, hidden_size=hidden_size,
                           output_size=output_size)
        _, _mape = network.train(x=X, y=y, lr=lr, max_epochs=max_epochs, verbosity=verbosity)
        all_epochs.append(max_epochs)
        mapes_sum += _mape        
        all_mapes.append(_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 100/100, Loss: 109.353590
MAPE: 0.740424
TRAINING FINISHED
Epoch 100/100, Loss: 105.008567
MAPE: 0.716828
TRAINING FINISHED
Epoch 100/100, Loss: 99.109856
MAPE: 0.694986
TRAINING FINISHED
Epoch 100/100, Loss: 90.074095
MAPE: 0.665925
TRAINING FINISHED
Epoch 100/100, Loss: 120.463195
MAPE: 0.774339
TRAINING FINISHED
Epoch 200/200, Loss: 32.480090
MAPE: 0.385493
TRAINING FINISHED
Epoch 200/200, Loss: 51.051703
MAPE: 0.484764
TRAINING FINISHED
Epoch 200/200, Loss: 58.530814
MAPE: 0.525577
TRAINING FINISHED
Epoch 200/200, Loss: 42.158746
MAPE: 0.431681
TRAINING FINISHED
Epoch 200/200, Loss: 57.349810
MAPE: 0.514594
TRAINING FINISHED
Epoch 300/300, Loss: 13.919602
MAPE: 0.222992
TRAINING FINISHED
Epoch 300/300, Loss: 21.071886
MAPE: 0.277742
TRAINING FINISHED
Epoch 300/300, Loss: 12.594759
MAPE: 0.202323
TRAINING FINISHED
Epoch 300/300, Loss: 19.153414
MAPE: 0.268649
TRAINING FINISHED
Epoch 300/300, Loss: 18.198392
MAPE: 0.251069
TRAINING FINISHED
Epoch 400/400, Loss

In [9]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_epochs, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=epochs, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от количества итераций обучения',
    font=dict(size=20),
    xaxis=dict(title='Количество итераций обучения'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [10]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
hidden_sizes = list(range(2, 22, 2))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 20
window_size = 5
batch_size = 2
output_size = 3

verbosity = 100000
max_mape = 1e-3
learn_until_meet_mape = True
lr = 1e-4
max_epochs = 250

sequence = list(arithmetic_progression(seq_length, a0=1, d=1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)
y = y

for size in hidden_sizes:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):              
        
        network = GRUModel(input_size=window_size, hidden_size=size,
                           output_size=output_size)
        _, _mape = network.train(x=X, y=y, lr=lr, max_epochs=max_epochs, verbosity=verbosity)
        all_sizes.append(size)
        mapes_sum += _mape        
        all_mapes.append(_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 250/250, Loss: 190.641724
MAPE: 0.982305
TRAINING FINISHED
Epoch 250/250, Loss: 94.860290
MAPE: 0.671234
TRAINING FINISHED
Epoch 250/250, Loss: 116.757317
MAPE: 0.751608
TRAINING FINISHED
Epoch 250/250, Loss: 68.173592
MAPE: 0.542227
TRAINING FINISHED
Epoch 250/250, Loss: 167.770677
MAPE: 0.921551
TRAINING FINISHED
Epoch 250/250, Loss: 65.963469
MAPE: 0.537713
TRAINING FINISHED
Epoch 250/250, Loss: 102.345486
MAPE: 0.688813
TRAINING FINISHED
Epoch 250/250, Loss: 112.906397
MAPE: 0.742418
TRAINING FINISHED
Epoch 250/250, Loss: 62.663889
MAPE: 0.537882
TRAINING FINISHED
Epoch 250/250, Loss: 124.408515
MAPE: 0.769038
TRAINING FINISHED
Epoch 250/250, Loss: 65.132919
MAPE: 0.546919
TRAINING FINISHED
Epoch 250/250, Loss: 86.164594
MAPE: 0.641830
TRAINING FINISHED
Epoch 250/250, Loss: 51.116893
MAPE: 0.474815
TRAINING FINISHED
Epoch 250/250, Loss: 46.139171
MAPE: 0.443278
TRAINING FINISHED
Epoch 250/250, Loss: 71.078085
MAPE: 0.570272
TRAINING FINISHED
Epoch 250/250, L

In [11]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=hidden_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>количества нейронов на скрытом слое',
    font=dict(size=20),
    xaxis=dict(title='Количество нейронов на скрытом слое'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [12]:
# MAPE relation with sliding window
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()


all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 50
window_sizes = list(range(1, seq_length // 4))
hidden_size = 10
batch_size = 2
output_size = 3

verbosity = 100000
max_mape = 1e-3
learn_until_meet_mape = True
lr = 1e-4
max_epochs = 1000

sequence = list(arithmetic_progression(seq_length, a0=1, d=1))

for size in window_sizes:
    X, y = create_sliding_window_batches(sequence, size, batch_size, output_size)    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):              
        
        network = GRUModel(input_size=size, hidden_size=hidden_size,
                           output_size=output_size)
        _, _mape = network.train(x=X, y=y, lr=lr, max_epochs=max_epochs, verbosity=verbosity)
        all_sizes.append(size)
        mapes_sum += _mape        
        all_mapes.append(_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 1000/1000, Loss: 25.007611
MAPE: 0.274241
TRAINING FINISHED
Epoch 1000/1000, Loss: 32.668452
MAPE: 0.288787
TRAINING FINISHED
Epoch 1000/1000, Loss: 30.091497
MAPE: 0.281959
TRAINING FINISHED
Epoch 1000/1000, Loss: 32.787266
MAPE: 0.292049
TRAINING FINISHED
Epoch 1000/1000, Loss: 31.674165
MAPE: 0.298834
TRAINING FINISHED
Epoch 1000/1000, Loss: 28.908725
MAPE: 0.278214
TRAINING FINISHED
Epoch 1000/1000, Loss: 23.092873
MAPE: 0.243748
TRAINING FINISHED
Epoch 1000/1000, Loss: 26.283899
MAPE: 0.258263
TRAINING FINISHED
Epoch 1000/1000, Loss: 27.001250
MAPE: 0.261172
TRAINING FINISHED
Epoch 1000/1000, Loss: 24.416334
MAPE: 0.248995
TRAINING FINISHED
Epoch 1000/1000, Loss: 17.967296
MAPE: 0.208247
TRAINING FINISHED
Epoch 1000/1000, Loss: 18.436734
MAPE: 0.206682
TRAINING FINISHED
Epoch 1000/1000, Loss: 21.818226
MAPE: 0.222598
TRAINING FINISHED
Epoch 1000/1000, Loss: 20.450953
MAPE: 0.224416
TRAINING FINISHED
Epoch 1000/1000, Loss: 25.657194
MAPE: 0.245042
TRAINING F

In [13]:
# Plotting results
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=window_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>размера скользящего окна',
    font=dict(size=20),
    xaxis=dict(title='Размер скользящего окна'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [39]:
# Arithmetic
seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(arithmetic_progression(seq_length, 1, 1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")  

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.032208
MAPE: 0.012176
TEST
Предсказания на тестовой выборке:
 [18.21112006 19.15522951 19.99779925]
Истинное значение на тестовой выборке:
 [18 19 20]
MAPE: 0.6670%


In [43]:
# Fibonacci
seq_length = 15
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(fibonacci_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")

TRAINING FINISHED
Epoch 2000/2000, Loss: 705.390526
MAPE: 0.538357
TEST
Предсказания на тестовой выборке:
 [111.92968859 181.08958216 293.08412494]
Истинное значение на тестовой выборке:
 [144 233 377]
MAPE: 22.2697%


In [37]:
# 1 / n
seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(one_by_n_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.001617
MAPE: 0.307867
TEST
[0.07692308 0.07142857 0.06666667 0.0625     0.05882353]
Предсказания на тестовой выборке:
 [0.07368016 0.06275571 0.0771424 ]
Истинное значение на тестовой выборке:
 [0.05555556 0.05263158 0.05      ]
MAPE: 35.3816%


In [44]:
# 1, -1, 1, -1, ...
seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(plus_one_minus_one_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%") 

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.079403
MAPE: 0.191956
TEST
Предсказания на тестовой выборке:
 [-0.39585527  0.15670909 -0.18489909]
Истинное значение на тестовой выборке:
 [-1  1 -1]
MAPE: 75.4179%


In [ ]:
# n ** 2
seq_length = 10
window_size = 3
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(squared_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%") 

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.282809
MAPE: 0.009512
TEST
Предсказания на тестовой выборке:
 [ 64.34795357  81.2383435  100.33545482]
Истинное значение на тестовой выборке:
 [ 64  81 100]
MAPE: 0.3911%
